# Linear regressionn convertor

In [ ]:
import numpy as np
import onnx
from onnx.helper import (make_model, make_node, make_graph, make_tensor_value_info)
from onnx import numpy_helper, TensorProto

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

# train the data
iris = load_iris()
X, y = iris.data, iris.target
X = X.astype(np.float32)
X_train, X_test, y_train, y_test = train_test_split(X, y)
model = LinearRegression()
model.fit(X_train, y_train)

In [ ]:
def linear_to_onnx(model): 
    # initializer
    A = numpy_helper.from_array(model.coef_, name = 'A') # coefficient
    B = numpy_helper.from_array(model.intercept_, name = 'B') # intercept
    
    # inputs
    X = make_tensor_value_info('X', TensorProto.FLOAT, [None, None])

    # outputs
    Y = make_tensor_value_info('Y', TensorProto.FLOAT, [None])

    # nodes
    node1 = make_node('MatMul', ['X', 'A'], ['XA'])
    node2 = make_node('Add', ['XA', 'B'], ['Y'])

    graph = make_graph([node1, node2],  # nodes
                        'lr',  # name
                        [X], # input
                        [Y], # output
                        [A, B]) # initializer

    # create onnx model    
    onnx_model = make_model(graph)
    
    # version controll
    onnx_model.ir_version = 9
    del onnx_model.opset_import[:]
    opset = onnx_model.opset_import.add()
    opset.version = 20

    return onnx_model

In [ ]:
onnx_model = linear_to_onnx(model)

with open("linear_regression.onnx", "wb") as f:
     f.write(onnx_model.SerializeToString())

In [ ]:
model.predict(X_test)

In [ ]:
# Compute the prediction with onnxruntime.
import onnxruntime as rt

sess = rt.InferenceSession('linear_regression.onnx', providers=["CPUExecutionProvider"])
sess.run(None, {"X": X_test})

In [ ]:
if __name__ == "__main__":
    # Path to the ONNX model file
    onnx_file_path = "linear_model.onnx"
    
    # Example input data (replace with actual data)
    input_data = np.array([[10]], dtype=np.float32)
    
    # Create an inference session
    session = create_onnx_session(onnx_file_path)
    
    # Perform inference
    result = linear_regression_inference(session, input_data)
    print("Inference result:", result)

## Create own backend runtime

In [ ]:
import onnx
# load ONNX model
linear_model = onnx.load("linear_model.onnx")

In [ ]:
import onnx2tf

# onnx2tf.convert()

In [ ]:
linear_model

In [ ]:
start = 0
end = 0
operators = []


model_name = linear_model


while start != -1:
    start = str(model_name.graph.node).find('op_type: "', end)
    end = str(model_name.graph.node).find('\n', start)
    operators.append(str(model_name.graph.node)[start:end][10:-1])


operators != operators.pop()
operators

In [ ]:
# get onnx operators

start = 0
end = 0
operators = []

model_name = linear_model

while start != -1: 
    start = str(model_name.graph.node).find('op_type: "', end)
    end = str(model_name.graph.node).find('\n', start)
    operators.append(str(model_name.graph.node)[start:end][10:-1])

operators != operators.pop()
operators

In [ ]:
# Define a dictionary mapping keys to functions

def MatMul(n, m):
    return np.dot(n, m)

def Add(n, m):
    return n+m

function_dict = {
    "MatMul": MatMul,
    "Add": Add
    }

In [ ]:
def onnx_run(model, input):
    # get onnx operators

    start = 0
    end = 0
    operators = []

    model_name = model

    while start != -1: 
        start = str(model_name.graph.node).find('op_type: "', end)
        end = str(model_name.graph.node).find('\n', start)
        operators.append(str(model_name.graph.node)[start:end][10:-1])

    operators != operators.pop()

    # run operators

    from onnx import numpy_helper

    result = input

    for i in range(len(operators)): 
        result = function_dict[operators[i]](result, numpy_helper.to_array(model.graph.initializer[i]))
    
    return result

In [ ]:
onnx_run(linear_model, 10)

In [ ]:
model.predict(X_test)

## Run model with onnx backend

In [ ]:
from onnx.backend.base import Backend


Backend.run_model(model = onnx_model, inputs = [6.4, 2.7, 5.3, 1.9]) 

# CNN converter

In [ ]:
np.array([1, 2, 3])

In [ ]:
import numpy as np

# initializer
value = np.array([1/255], dtype = np.float32)
rescale_unit = numpy_helper.from_array(value, name = 'rescale_unit')

reshape_value = np.array([-1, 30976], dtype = np.float32)
reshape_value = numpy_helper.from_array(reshape_value, name = 'reshape_value')

# tensor value
input = make_tensor_value_info('input', 
                               TensorProto.FLOAT, 
                               [1, 180, 180, 3])

output = make_tensor_value_info('output', 
                                TensorProto.FLOAT, 
                                [None])

# nodes
rescale_node = make_node('MatMul', ['input', 'rescale_unit'], ['rescaled_input'])

transpose_node = make_node('Transpose', ['rescaled_input'], ['transpose_input'], perm = [0, 3, 1, 2])
# permutation: [1, 180, 180, 3] ---(perm = [0, 3, 1, 2])--> [1, 3, 180, 180]

conv_node = make_node(
        'Conv',
        ['transpose_input', 'weights'],
        ['conv_output'],
        name = 'conv',
        kernel_shape = [3, 3],
        strides = [1, 1],
        pads = [1, 1, 1, 1]
        )

relu_node = make_node('Relu', ['conv_output'], ['relu_output'])

maxpool_node = make_node(
        'MaxPool', 
        ['relu_output'], 
        ['maxpool_output'], 
        name = 'maxpool', 
        kernel = [2, 2], 
        strides = [2, 2]
    )

reshape_node = make_node(
        'Reshape', 
        ['maxpool_output', 'reshape_value'], 
        ['output']
)

# create graph
graph = make_graph([rescale_node, transpose_node, conv_node, relu_node, maxpool_node, reshape_node], 
                   'cnn', 
                   [input], 
                   [output], 
                   [rescale_unit, reshape_value]
                   )

onnx_model = make_model(graph)
with open("naive_cnn.onnx", "wb") as f:
     f.write(onnx_model.SerializeToString())

In [ ]:
def cnn_too_onnx(model, input_shape): 
    # input 
    input = make_tensor_value_info('input', 
                                   TensorProto.FLOAT, 
                                   input_shape)

    # output
    output = make_tensor_value_info('output', 
                                    TensorProto.FLOAT, 
                                    [None])
    
    # create node for convolutional filter
    conv_node = make_node(
        'Conv',
        ['input', 'weights'],
        ['conv_output'],
        name = 'conv',
        kernel_shape = [3, 3],
        strides = [1, 1],
        pads = [1, 1, 1, 1]
        )
    
    # create node for relu activation
    relu_node = make_node(
        'Relu', 
        ['conv_output'], 
        ['relu_output'], 
        name = 'relu'
    )
    
    # create node for max pooling
    maxpool_node = make_node(
        'MaxPool', 
        ['relu_output'], 
        ['output'], 
        name = 'maxpool', 
        kernel = [2, 2], 
        strides = [2, 2]
    )

    graph = make_graph(
        [conv_node, relu_node, maxpool_node], 
        'cnn_model', 
        [input], 
        [output]
    )

    onnx_model = make_model(graph)

    # version controll
    onnx_model.ir_version = 9
    del onnx_model.opset_import[:]
    opset = onnx_model.opset_import.add()
    opset.version = 20
    
    return onnx_model

In [ ]:
cnn_onnx = cnn_too_onnx()
onnx.save(cnn_onnx, 'simple_cnn.onnx')

In [ ]:
import onnx
simple_cnn = onnx.load('sequential_2.onnx')

In [ ]:
start = 0
end = 0
operators = []

model_name = simple_cnn

while start != -1: 
    start = str(model_name.graph.node).find('op_type: "', end)
    end = str(model_name.graph.node).find('\n', start)
    operators.append(str(model_name.graph.node)[start:end][10:-1])

operators != operators.pop()
operators

In [ ]:
simple_cnn

In [ ]:
simple_cnn.graph.initializer[2], numpy_helper.to_array(simple_cnn.graph.initializer[2])

In [ ]:
import tensorflow as tf
flower_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/4/4e/Bridal_pink_-_morwell_rose_garden.jpg/800px-Bridal_pink_-_morwell_rose_garden.jpg"
flower_path = tf.keras.utils.get_file(origin = flower_url)

img = tf.keras.utils.load_img(
    flower_path, target_size = (180, 180)
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

In [ ]:
result1 = function_dict['MatMul'](img_array, numpy_helper.to_array(simple_cnn.graph.initializer[1]))
result2 = function_dict['Add'](result1, numpy_helper.to_array(simple_cnn.graph.initializer[0]))
result2

In [ ]:
# Define a dictionary mapping keys to functions

def MatMul(input, m):
    return np.dot(input, m)

def Add(input, m):
    return input+m

def Transpose(input): 
    transposed_matrix = [[input[j][i] for j in range(len(input))] for i in range(len(input[0]))]
    return transposed_matrix

def Convolutional(input, kernel): 
    m, _ = kernel.shape

    y, x = input.shape
    y = y - m + 1
    x = x - m + 1

    output_layer = np.zeros((y,x))

    for i in range(y):
        for j in range(x):
            output_layer[i][j] = np.sum(input[i:i + m, j:j + m] * kernel)

    return output_layer

def Relu(input):
    return np.maximum(0, input)

def MaxPool(input, pool_size):

    m, n = input.shape
    pool_height, pool_width = pool_size

    output_height = m // pool_height
    output_width = n // pool_width
    output_layer = np.zeros((output_height, output_width))

    for i in range(output_height):
        for j in range(output_width):
            start_i = i * pool_height
            start_j = j * pool_width
            max_val = np.max(input[start_i:start_i + pool_height,
                                         start_j:start_j + pool_width])
            output_layer[i, j] = max_val

    return output_layer

def Reshape(array, new_shape):

    new_size = 1
    for dim in new_shape:
        new_size *= dim
    
    reshaped_array = []
    current_index = 0
    
    for i in range(new_shape[0]):
        row = []
        for j in range(new_shape[1]):
            row.append(array[current_index])
            current_index += 1
        reshaped_array.append(row)
    
    return reshaped_array

def Gemm(input, weight, bias):
    return np.dot(input, weight) + bias
    
function_dict = {
    "MatMul": MatMul,
    "Add": Add, 
    "Transpose": Transpose, 
    "Conv": Convolutional, 
    "Relu": Relu, 
    "MAxPool": MaxPool, 
    "Reshape": Reshape, 
    "Gemm": Gemm
    }

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
import tensorflow as tf

model = Sequential([
  layers.Rescaling(1./255, input_shape=(180, 180, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(5)
])

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
operators = []

for layer in model.layers: 
    operators.append(layer._name)

operators

# Random forest convertor

In [ ]:
import numpy as np
import onnx
from onnx import helper

# Define a function to create a single decision tree node
def make_decision_tree_node(tree_index, feature_index, threshold, left_child, right_child, leaf_value):
    node_inputs = [
        f'input_{tree_index}', f'feature_index_{tree_index}', f'threshold_{tree_index}',
        f'left_child_{tree_index}', f'right_child_{tree_index}', f'leaf_value_{tree_index}'
    ]
    
    node_outputs = [f'output_{tree_index}']
    
    return helper.make_node(
        'DecisionTree',
        node_inputs,
        node_outputs,
        name=f'decision_tree_{tree_index}',
        domain="ai.onnx.ml"
    )

# Define the random forest model
def make_model(num_trees, num_features):
    graph_inputs = [
        helper.make_tensor_value_info('input', onnx.TensorProto.FLOAT, (None, num_features))
    ]
    
    graph_outputs = [
        helper.make_tensor_value_info('output', onnx.TensorProto.FLOAT, (None,))
    ]
    
    nodes = []
    for i in range(num_trees):
        # Example decision tree parameters (replace with your actual parameters)
        feature_index = np.random.randint(0, num_features)
        threshold = np.random.rand()
        left_child = np.random.randint(-1, num_features)
        right_child = np.random.randint(-1, num_features)
        leaf_value = np.random.rand()
        
        nodes.append(
            make_decision_tree_node(i, feature_index, threshold, left_child, right_child, leaf_value)
        )
    
    return helper.make_model(
        helper.make_graph(
            nodes,
            'random_forest_model',
            graph_inputs,
            graph_outputs
        ),
        producer_name='random_forest'
    )

# Create the model
num_trees = 5
num_features = 10
model = make_model(num_trees, num_features)

# Save the model to an ONNX file
onnx_path = "random_forest.onnx"
onnx.save(model, onnx_path)
print(f"Model exported to {onnx_path}")
